<a href="https://colab.research.google.com/github/emoncse4/Neural-Network-SMS-Text-Classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load datasets
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'text'])

# Convert labels to numbers: ham = 0, spam = 1
train_df['type'] = train_df['type'].map({'ham': 0, 'spam': 1})
test_df['type'] = test_df['type'].map({'ham': 0, 'spam': 1})

train_labels = train_df.pop('type').values
test_labels = test_df.pop('type').values

train_ds = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['text'].values, test_labels))

VOCAB_SIZE = 1000
MAX_LEN = 100

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_LEN
)

# Train the layer on our training text
vectorize_layer.adapt(train_df['text'].values)



In [ ]:
# Prepare data
train_texts = train_df['text'].astype(str).values
test_texts = test_df['text'].astype(str).values

# Increase VOCAB and MAX_LEN for better coverage
VOCAB_SIZE = 1000
MAX_LEN = 250

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_LEN
)
vectorize_layer.adapt(train_texts)

# Build a more powerful LSTM model
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(VOCAB_SIZE, 64),
    keras.layers.Bidirectional(keras.layers.LSTM(32)), # Understands word order
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train for 10 epochs (LSTMs learn faster)
model.fit(
    train_texts,
    train_labels,
    epochs=10,
    validation_data=(test_texts, test_labels),
    verbose=1
)

In [ ]:
def predict_message(pred_text):
    # Wrap input in a numpy array
    input_data = np.array([pred_text], dtype=object)

    # Get probability
    probability = model.predict(input_data, verbose=0)[0][0]

    # Classification
    label = "ham" if probability < 0.5 else "spam"

    return [float(probability), label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
